In [1]:
import sys
sys.path.append('../../../')
import warnings
warnings.filterwarnings(action='ignore')
import torch
from torch import nn
from train import val        

from static_definer import *

In [2]:
from models.domain_shift.adversarial.functions import DomainDiscriminator, train
from models.bisenet.build_bisenet import BiSeNet

In [3]:

# # defining a CrossEntropyLoss for the segmentation and a BCEWithLogitsLoss for the domain classification
generator_loss = nn.CrossEntropyLoss(ignore_index=19)
discriminator_loss = nn.BCEWithLogitsLoss()

# defining the models
generator = BiSeNet(num_classes=num_classes,context_path='resnet18',with_interpolation=True)

# generator.load_state_dict(torch.load('/kaggle/working/bisenetTrainingWeights/best_bisenet_model_weights.pth'))

# defining the Discriminator
discriminator = DomainDiscriminator(num_classes=num_classes,with_grl=False)

# defining the optimizer
generator_optimizer = torch.optim.SGD(generator.parameters(), lr=1e-2, momentum=0.9, weight_decay=5e-4)
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=1e-3)

In [4]:
from callbacks import WandBCallback, PrettyPrintCallback

# defining the callbacks
callbacks = [
    WandBCallback(project_name='MLDL_PART4_v1',
                  run_name='bisenet_domain_shift_v1'),
]

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/Users/sinabehnam/Desktop/Projects/Polito/ML & Deep/Project/MLDL2024_Project_Group_2/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /Users/sinabehnam/.netrc


In [6]:
for epoch in range(1, 10):

    train(iterations=100,
          epoch=epoch,
          generator=generator,
          discriminator=discriminator,
          generator_optimizer=generator_optimizer,
          discriminator_optimizer=discriminator_optimizer,
          generator_loss=generator_loss,
          discriminator_loss=discriminator_loss,
          source_dataloader=train_dataloaderGTA5,
          target_dataloader=train_dataloader,
          device=device,
          lambda_=1e-4,
          callbacks=callbacks
          )

    if epoch % 5 == 0:
        print('-'*50)
        val(epoch, generator, val_dataloader, num_classes, device, callbacks=callbacks)
        print('-'*50)

for callback in callbacks:
    callback.on_train_end()

torch.save(generator.state_dict(), f'generator_{epoch}.pth')
torch.save(discriminator.state_dict(), f'discriminator_{epoch}.pth')
    

Epoch 1:   0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 